In [37]:
import pandas as pd
import os
import json
import numpy as np
import re

In [38]:
## reading the Survey File
#surveyData = pd.read_csv("C:\Projects\MSS\Loading _AmenityData_from_Survey\Test.csv",sep="," , skiprows=1)
surveyData = pd.read_csv("C:\Projects\MSS\Loading _AmenityData_from_Survey\AmenityClubSurvey FINAL v2 Season Cleaned_deletedFirstTwoColumn.csv",sep="," , skiprows=1)
surveyData.head()

## Reading the Config File and Extracting the Information from there
config_file = open("C:\Projects\MSS\Loading _AmenityData_from_Survey\Python_Codes\ClubCorp\TennisCourts\TennisCourtsColMapping.json")
tennisCourts_config = json.load(config_file)

columnName = ""

In [39]:
def addToDayTimeMap (phonebook, key, value):
    if key in phonebook:
        x =  phonebook[key]
        phonebook[key] = x + "," + value
        
    else :
        phonebook[key] = value

In [40]:
def prepareHoursOfOperationJson(rowIndexes,hoursOfOperationInfoTennis, hoursOfOperationInfoTennisProShop, surveyData):
    
    columnIndexForMondayOpenTime = tennisCourts_config['columnIndexForMondayOpenTime']
    hoursOfOperationInfo = ""
    
    allHoursOfOperation = []
    
    for i in rowIndexes : 
        
        mondayOpenTime = surveyData.iat[i, columnIndexForMondayOpenTime]
        
        if pd.notnull(mondayOpenTime) and  mondayOpenTime != "same":
           hoursOfOperationInfo = hoursOfOperationInfoTennis;
        else :
           hoursOfOperationInfo = hoursOfOperationInfoTennisProShop; 
        
        
        hoursOfOperationForEachRow = []
        hoursOfOperation = {}
        hoursOfOperationValue = {}

        season = "All Season"
        ## adding season value
        if season and not pd.isnull(season):
            seasonValue = {}
            seasonValue["value"] = season
            hoursOfOperationValue["Season"] = [seasonValue]
        
        
        ## creating day time map so that days with same open and closed time can handled properly
        dayTimeMap = {}    
        for j in range(0, len(hoursOfOperationInfo)):
            
            dayInfo = hoursOfOperationInfo[j]
            day = dayInfo["Day"]
            openTimeIndex = dayInfo["OpenTime"]
            closeTimeIndex = dayInfo["CloseTime"]
                
            startTimeRaw = surveyData.iat[i, openTimeIndex]
            endTimeRaw = surveyData.iat[i, closeTimeIndex]
            
            startTime = ""
            endTime = ""
            if pd.notnull(startTimeRaw):
                startTime = str(startTimeRaw).strip()
            
            if pd.notnull(endTimeRaw):
                endTime = str(endTimeRaw).strip()
                
            
            
            if startTime == "Closed" or endTime == "Closed" :
                continue
            else :  
                addToDayTimeMap(dayTimeMap, startTime + "#$" + endTime, day)
            
        
        dayAndTimingArray = []
        
        for startAndEndTime, days in dayTimeMap.items():
            
            startTime,endTime = startAndEndTime.split("#$")
            openDays = days.split(",")
            ## Adding Day and Timing
            dayAndTiming = {}
            dayAndTimingValue = {}
            for day in openDays :
                dayValue = {}
                dayValue["value"] = True
                dayAndTimingValue[day] = [dayValue]

            timing = {}
            timingValue = {}

            if startTime and not pd.isnull(startTime): 
                startTimeDict ={}
                startTimeDict["value"] = startTime
                timingValue["StartTime"] =  [startTimeDict]

            if endTime and not pd.isnull(endTime):
                endTimeDict ={}
                endTimeDict["value"] = endTime
                timingValue["EndTime"] =  [endTimeDict]
            
            ## Adding day part as "All Day"
            dayPart ={}
            dayPart["value"] = "All Day"
            timingValue["DayPart"] =  [dayPart]


            if timingValue :
                timing["value"] = timingValue
                dayAndTimingValue["Timing"] = [timing]

            if dayAndTimingValue : 
                dayAndTiming["value"] = dayAndTimingValue    
                dayAndTimingArray.append(dayAndTiming)
        
        hoursOfOperationValue["DayAndTiming"] = dayAndTimingArray
        if hoursOfOperationValue :
            hoursOfOperation["value"] = hoursOfOperationValue
            
        hoursOfOperationForEachRow.append(hoursOfOperation)
        allHoursOfOperation.append(json.dumps(hoursOfOperationForEachRow))
    
    return allHoursOfOperation

In [41]:
def determineBooleanValues (dataFrame):
    val = False
    if pd.notnull(dataFrame[columnName]) :
        if dataFrame[columnName] == columnName:
            val = True
    return val

In [42]:
## Getting the output File name
output_fileName = tennisCourts_config['outputFileName']

rowIndexForTennisCourt = []
rowIndexForClubsNotHavingTennisCourts = []

## Getting the values of Number of Tennis Course
num_of_tennis_course_columnIndex = tennisCourts_config['columnIndexOfNumOfTennisCourse']
tennisCourtCountInformation = surveyData.iloc[:,num_of_tennis_course_columnIndex]

for i in range(0,len(tennisCourtCountInformation)):
    tennisCourtCount = tennisCourtCountInformation[i]
    
    if tennisCourtCount>=1 :
        rowIndexForTennisCourt.append(i)
    else :
        rowIndexForClubsNotHavingTennisCourts.append(i)

all_column_names = []
tennis_court_related_column_names = []

columnIndexForTennisCourts = []

common_column_count = len(tennisCourts_config['CommonColumn'])

for i in range(0, common_column_count):
    columnIndexForTennisCourts.append(tennisCourts_config['CommonColumn'][i]['columnIndex'])
    all_column_names.append(tennisCourts_config['CommonColumn'][i]['outputColumnName'])

tennisCourts_column_count = len(tennisCourts_config['TennisCourtsRelatedColumn'])

for j in range(0, tennisCourts_column_count):
    columnIndexForTennisCourts.append(tennisCourts_config['TennisCourtsRelatedColumn'][j]['columnIndex'])
    all_column_names.append(tennisCourts_config['TennisCourtsRelatedColumn'][j]['outputColumnName'])
    tennis_court_related_column_names.append(tennisCourts_config['TennisCourtsRelatedColumn'][j]['outputColumnName'])
    
## getting Hours of Related column information from config file
hoursOfOperationInfoTennis = tennisCourts_config['HoursOfOperationInformation']
hoursOfOperationInfoTennisProShop = tennisCourts_config['TennisProShopHoursOfOperation']


## Data for Tennis Courts
tennisCourts = surveyData.iloc[rowIndexForTennisCourt, columnIndexForTennisCourts]
tennisCourts.columns = all_column_names

tennisCourts['Index']=1

hrsOfOperation = prepareHoursOfOperationJson(rowIndexForTennisCourt,hoursOfOperationInfoTennis, hoursOfOperationInfoTennisProShop, surveyData)
tennisCourts['HrsOfOpeartion'] = hrsOfOperation

tennisCourts['Status'] = 'Active'
tennisCourts['Type'] = 'Tennis & Racquet Sports'
tennisCourts['Name'] = 'Tennis Courts'

for x  in tennis_court_related_column_names:
    columnName = x
    tennisCourts[columnName + '_Boolean'] = tennisCourts.apply(determineBooleanValues, axis=1)


tennisCourts.to_csv("C:/Projects/MSS/Loading _AmenityData_from_Survey/Python_Codes/ClubCorp/TennisCourts/" + output_fileName)


## Creating file for those clubs that does not have any tennis Courts
clubsThatDoesNotHaveTennisCourts = surveyData.iloc[rowIndexForClubsNotHavingTennisCourts, columnIndexForTennisCourts]
clubsThatDoesNotHaveTennisCourts.to_csv("C:/Projects/MSS/Loading _AmenityData_from_Survey/Python_Codes/ClubCorp/TennisCourts/TennisCourts_ClubNotHaving.csv")
